In [11]:
import cv2
import numpy as np

In [13]:
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# Background subtractor (more stable background, ignores shadows)
fgbg = cv2.createBackgroundSubtractorMOG2(
    history=100,
    varThreshold=64,
    detectShadows=True
)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))

def merge_boxes(boxes, overlapThresh=0.4):
    """Merge overlapping bounding boxes (Non-Maximum Suppression)."""
    if len(boxes) == 0:
        return []
    boxes = np.array(boxes)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 0] + boxes[:, 2]
    y2 = boxes[:, 1] + boxes[:, 3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    pick = []

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last],
                                               np.where(overlap > overlapThresh)[0])))
    merged = boxes[pick].astype("int")
    return merged

while True:
    success, img = cap.read()
    if not success:
        break
    img = cv2.flip(img, 1)

    # Apply background subtractor slowly (stable background)
    fgmask = fgbg.apply(img, learningRate=0.001)

    # Keep only pure foreground pixels (drop shadows)
    fgmask = np.where(fgmask == 255, 255, 0).astype('uint8')

    # Clean noise with blur + morphology
    fgmask = cv2.medianBlur(fgmask, 5)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel, iterations=2)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    boxes = []
    for c in contours:
        area = cv2.contourArea(c)
        if area > 2000:  # filter out small blobs
            x, y, w, h = cv2.boundingRect(c)
            boxes.append((x, y, w, h))

    # Merge overlapping boxes
    merged_boxes = merge_boxes(boxes, overlapThresh=0.4)

    # Filter nested boxes (remove boxes fully inside others)
    filtered = []
    for i, (x1, y1, w1, h1) in enumerate(merged_boxes):
        inside = False
        for j, (x2, y2, w2, h2) in enumerate(merged_boxes):
            if i != j:
                if (x1 > x2 and y1 > y2 and
                    x1 + w1 < x2 + w2 and
                    y1 + h1 < y2 + h2):
                    inside = True
                    break
        if not inside:
            filtered.append((x1, y1, w1, h1))

    # Draw final rectangles
    for (x, y, w, h) in filtered:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow('Webcam', img)
    cv2.imshow('Mask', fgmask)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()